#### Monday, January 13, 2025

# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p> 
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [1]:
# Install the requirements in Google Colab
# !pip install transformers datasets trl huggingface_hub

# Authenticate to Hugging Face

from huggingface_hub import login
login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

Token has not been saved to git credential helper.


In [2]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)
device

'cuda'

In [3]:
# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

In [4]:
# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

In [5]:
# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

# Generate with the base model

Here we will try out the base model which does not have a chat template. 

In [6]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

In [7]:
# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)
print(formatted_prompt)

<|im_start|>user
Write a haiku about programming<|im_end|>



In [8]:
# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`,.

In [9]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

In [10]:
# TODO: 🦁 If your dataset is not in a format that TRL can convert to the chat template, you will need to process it. Refer to the [module](../chat_templates.md)

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [11]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model
)

/home/rob/Data/Documents/Github/rkaunismaa/smol-course/.venv/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

/home/rob/Data/Documents/Github/rkaunismaa/smol-course/.venv/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/2260 [00:00<?, ? examples/s]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [13]:
# This places a huge load on the GPU ... 97% usage ... 5032MiB used!
# Train the model
trainer.train()

# 4m 32.3s

  0%|          | 0/1000 [00:00<?, ?it/s]

{'loss': 1.6774, 'grad_norm': 2.7205331325531006, 'learning_rate': 4.9500000000000004e-05, 'epoch': 0.02}
{'loss': 1.2563, 'grad_norm': 2.644493579864502, 'learning_rate': 4.9e-05, 'epoch': 0.04}
{'loss': 1.234, 'grad_norm': 2.3947083950042725, 'learning_rate': 4.85e-05, 'epoch': 0.05}
{'loss': 1.161, 'grad_norm': 2.5028371810913086, 'learning_rate': 4.8e-05, 'epoch': 0.07}
{'loss': 1.0657, 'grad_norm': 2.2998485565185547, 'learning_rate': 4.75e-05, 'epoch': 0.09}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.158981204032898, 'eval_runtime': 1.8303, 'eval_samples_per_second': 65.016, 'eval_steps_per_second': 8.195, 'epoch': 0.09}
{'loss': 1.1566, 'grad_norm': 2.2886159420013428, 'learning_rate': 4.7e-05, 'epoch': 0.11}
{'loss': 1.1648, 'grad_norm': 2.0124387741088867, 'learning_rate': 4.6500000000000005e-05, 'epoch': 0.12}
{'loss': 1.1244, 'grad_norm': 2.128488779067993, 'learning_rate': 4.600000000000001e-05, 'epoch': 0.14}
{'loss': 1.086, 'grad_norm': 2.224376916885376, 'learning_rate': 4.55e-05, 'epoch': 0.16}
{'loss': 1.1116, 'grad_norm': 2.0808310508728027, 'learning_rate': 4.5e-05, 'epoch': 0.18}


  0%|          | 0/15 [00:00<?, ?it/s]

...

  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.021671175956726, 'eval_runtime': 1.7935, 'eval_samples_per_second': 66.352, 'eval_steps_per_second': 8.364, 'epoch': 1.68}
{'loss': 0.8609, 'grad_norm': 1.8237701654434204, 'learning_rate': 2.0000000000000003e-06, 'epoch': 1.7}
{'loss': 0.7802, 'grad_norm': 1.7281394004821777, 'learning_rate': 1.5e-06, 'epoch': 1.72}
{'loss': 0.7957, 'grad_norm': 1.7798895835876465, 'learning_rate': 1.0000000000000002e-06, 'epoch': 1.73}
{'loss': 0.8345, 'grad_norm': 1.7968873977661133, 'learning_rate': 5.000000000000001e-07, 'epoch': 1.75}
{'loss': 0.7915, 'grad_norm': 1.764422059059143, 'learning_rate': 0.0, 'epoch': 1.77}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.0214849710464478, 'eval_runtime': 1.7872, 'eval_samples_per_second': 66.585, 'eval_steps_per_second': 8.393, 'epoch': 1.77}
{'train_runtime': 272.3049, 'train_samples_per_second': 14.689, 'train_steps_per_second': 3.672, 'train_loss': 0.9605711183547974, 'epoch': 1.77}


TrainOutput(global_step=1000, training_loss=0.9605711183547974, metrics={'train_runtime': 272.3049, 'train_samples_per_second': 14.689, 'train_steps_per_second': 3.672, 'total_flos': 587568496250880.0, 'train_loss': 0.9605711183547974, 'epoch': 1.7699115044247788})

In [14]:
# Save the model
trainer.save_model(f"./{finetune_name}")

In [15]:
# Nope! I am not gonna do this!
# trainer.push_to_hub(tags=finetune_tags)

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px; color:black'>
    <h2 style='margin: 0;color:blue'>Bonus Exercise: Generate with fine-tuned model</h2>
    <p>🐕 Use the fine-tuned to model generate a response, just like with the base example..</p>
</div>

In [16]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)

# TODO: use the fine-tuned to model generate a response, just like with the base example.

## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.